### Training Bayesian Neural Network
**Authors:** Akash Mahajan, Van-Hai Bui

In [ ]:
# Import packages
import pandas as pd
import numpy as np

from torch.nn import Softplus
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import GaussianNLLLoss

from utils.Datasets import TimeSeriesDataset
from utils.Network import NetConfig, TrainConfig
from utils.Priors import GaussianPrior, Prior
from utils.nn.BayesianNN import BayesianNN
from utils.Network import Trainer

#### 1. Importing dataset

In [ ]:
df = pd.read_csv("output/data.csv", parse_dates=True)

# setting the time index
df["timestamp"] = pd.to_datetime(df["timestamp"])
df.set_index("timestamp", inplace=True)
df.head()

In [ ]:
# Standardize only the continous feature variables, not categorical variables
from utils.data_loader import standardize

x_std_features = ['Interior_Zone_temp_mean', 'solar_radiation', 'relative_humidity', 'air_temperature', 'wind_speed']
std_features, _, _ = standardize(df[x_std_features])

df[x_std_features] = std_features
df.head()

In [ ]:
# two and half years of data for training and half years of data for testing 
start_train = "2018-01-01"
end_train = "2020-06-30"
start_test = "2020-07-01"
end_test = "2020-12-31"

# Demand
target = "elec_kW"
y_columns = [target]

x_columns = [c for c in df.columns if c != target]
print("Targets:", y_columns)
print("Input features:", x_columns)

In [ ]:
historical_sequence_length = 24
test_idx = df[start_test:end_test].index[historical_sequence_length:].to_numpy()

# Save test index for later use
np.save("output/test_index.npy", test_idx)

In [ ]:
# splitting training and testing datatset (Future development: validation data)
x_train = df[x_columns][start_train:end_train].values
y_train = df[y_columns][start_train:end_train].values
x_test = df[x_columns][start_test:end_test].values
y_test = df[y_columns][start_test:end_test].values

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# Training parameters
sequential = False
batch_size = 64
historical_sequence_length = 24

train_data = TimeSeriesDataset(x_train, y_train, historical_sequence_length, sequential)
# train_stats = train_data.transform()
train_loader = DataLoader(train_data, batch_size=batch_size, drop_last=False, shuffle=False, num_workers=0)

test_data = TimeSeriesDataset(x_test, y_test, historical_sequence_length, sequential)
# test_stats = test_data.transform()
test_loader = DataLoader(test_data, batch_size=batch_size, drop_last=False, shuffle=False, num_workers=0)

#### Training

In [ ]:
input_units = x_train.shape[-1] * historical_sequence_length
output_units = y_train.shape[-1]

prior = GaussianPrior(mean=0, spread=0.1)
net_config = NetConfig(prior, input_units, output_units, output_activation=Softplus(), model_name="BayesianNN")

print("Model input units:", input_units)
print("Model output units:", output_units)

In [ ]:
# Building the model
layer_units = [512, 512, 128]

for n_units in layer_units:
    net_config.add_layer(n_units)

model = BayesianNN(net_config)

In [ ]:
# Additional training parameters
epochs = 2000
learning_rate = 1e-4  # 0.0001

target_folder = f"Bayesian_ELEC_{epochs}_{learning_rate}_{batch_size}"

loss = GaussianNLLLoss(full=True, reduction='sum')
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=0)

train_config = TrainConfig(batch_size=batch_size, epochs=epochs, optimizer=optimizer, 
                           learning_rate=learning_rate, loss=loss, save_folder=target_folder)

In [ ]:
# Training
trainer = Trainer(train_loader, test_loader, model, train_config)
train_history = trainer.fit()   # This line automatically saves trained model

In [ ]:
# Save the training history
train_history.to_file(model_name=target_folder, config=train_config)